In [14]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier

from xgboost import XGBClassifier


from math import sqrt
import seaborn as sns
import matplotlib.pyplot as plt

In [15]:
phm_df = pd.read_parquet('../feature_extracts/', engine='fastparquet')


In [16]:
phm_df

,index,fault_class,individual,pdmp,pin,po,pdmp_avg_first_150,pdmp_avg_last_150,pdmp_avg_mid_300,pdmp_avg_diff,...,po_size,po_total_avg,po_variance,po_skew,po_kurtosis,po_twtt,pdmp_pin_kwut,pdmp_po_kwut,pin_po_kwut,pdmp_pin_po_kwht
0,22776,3,5,"[-0.194942, -0.187465, -0.180348, -0.173447, -...","[-1.731534, -1.755668, -1.738321, -1.691324, -...","[0.230999, 0.146074, 0.045333, -0.082227, -0.2...",-0.380677,0.166427,0.332923,-0.547104,...,571.0,-0.149746,1.056277,0.571217,-1.485157,-5.753815e-06,-1.673449,2.328661,4.145155,31.133930
1,22777,8,5,"[-0.246213, -0.273524, -0.308485, -0.348401, -...","[-1.553456, -1.535349, -1.530819, -1.498527, -...","[0.267242, 0.181471, 0.066593, -0.058671, -0.1...",-0.416614,0.107498,0.379532,-0.524112,...,571.0,-0.123757,1.060788,0.486906,-1.575010,4.370640e-07,-1.384548,2.107081,3.590630,28.394379
2,22778,4,5,"[0.276173, 0.256051, 0.233923, 0.207697, 0.179...","[-1.322575, -1.380218, -1.382825, -1.375426, -...","[0.470566, 0.445597, 0.413041, 0.369268, 0.310...",-0.490820,0.388297,0.206089,-0.879116,...,571.0,-0.122784,1.044903,0.499660,-1.561296,5.006699e-06,-1.479254,1.515997,3.062488,20.596146
3,22779,2,5,"[0.0658, 0.057669, 0.051469, 0.047736, 0.04593...","[-1.409725, -1.365621, -1.333474, -1.283352, -...","[-0.102403, -0.197452, -0.314519, -0.436627, -...",-0.793932,0.596038,0.544159,-1.389970,...,571.0,-0.008986,1.041080,-0.163733,-1.727554,-1.654965e-06,-1.272439,0.067728,1.342652,1.117999
4,22780,2,5,"[0.27768, 0.272525, 0.265093, 0.256061, 0.2469...","[-1.42767, -1.41073, -1.396357, -1.354666, -1....","[-0.0248, -0.119669, -0.252136, -0.41008, -0.5...",-0.804523,0.600944,0.530414,-1.405467,...,571.0,-0.007443,1.033000,-0.165585,-1.718661,-1.391775e-06,-1.125652,-0.045439,1.078964,0.833243
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34140,34141,1,6,"[-0.051017, -0.055426, -0.061033, -0.070764, -...","[-1.191121, -0.940782, -0.640797, -0.310252, -...","[-0.559606, -0.681762, -0.706164, -0.698716, -...",-0.319361,0.209585,0.343784,-0.528946,...,571.0,-0.113808,1.062574,0.484757,-1.594999,2.504394e-06,-1.847099,1.725427,3.675492,25.600899
34141,34142,1,6,"[0.027704, 0.00884, -0.005844, -0.01568, -0.02...","[-1.671334, -1.706622, -1.711954, -1.683148, -...","[0.378552, 0.316211, 0.229241, 0.119138, -0.00...",-0.225832,0.278967,0.148029,-0.504800,...,571.0,-0.115101,1.050273,0.501856,-1.560801,-1.586483e-06,-1.582425,1.751740,3.419522,23.742712
34142,34143,1,6,"[-0.037775, -0.053561, -0.073233, -0.097449, -...","[-1.728436, -1.737226, -1.713788, -1.683285, -...","[0.303343, 0.21896, 0.113885, -0.00383, -0.147...",-0.362291,0.274743,0.296034,-0.637034,...,571.0,-0.120584,1.053842,0.498116,-1.566424,-4.611677e-06,-1.557784,1.771423,3.414773,22.675776
34143,34144,1,6,"[0.095459, 0.086675, 0.078642, 0.068852, 0.057...","[-1.537821, -1.593619, -1.599173, -1.589957, -...","[0.420105, 0.371285, 0.301655, 0.215862, 0.128...",-0.401902,0.333803,0.261973,-0.735706,...,571.0,-0.115785,1.054297,0.475711,-1.586136,-3.706969e-06,-1.660925,1.669087,3.417492,19.866421


In [17]:
# Split the data into features (X) and target (y)
# X = phm_df.drop(columns=['pdmp', 'pin', 'po', 'fault_class', 'index', 'po_size', 'pdmp_size', 'pin_size'])
X = phm_df[['individual', 'pdmp_skew', 'pin_skew', 'po_skew', 'pdmp_variance', 'pin_variance', 'po_variance', 'pdmp_kurtosis', 'pin_kurtosis', 'po_kurtosis', 'pdmp_pin_po_kwht', 'pdmp_pin_kwut', 'pdmp_po_kwut', 'pin_po_kwut']]
# X = phm_df[['individual', 'pdmp_variance', 'pin_variance', 'po_variance']]
# X = phm_df[['individual', 'pdmp_total_avg', 'pin_total_avg', 'po_total_avg', 'pdmp_skew', 'pin_skew', 'po_skew']]

X = phm_df[['individual', \
#             'pdmp_twtt', 'pin_twtt', 'po_twtt', \
            'pdmp_pin_kwut', 'pdmp_po_kwut', 'pin_po_kwut', 'pdmp_pin_po_kwht']]
y = phm_df['fault_class']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train

,individual,pdmp_skew,pin_skew,po_skew,pdmp_variance,pin_variance,po_variance,pdmp_kurtosis,pin_kurtosis,po_kurtosis,pdmp_pin_po_kwht,pdmp_pin_kwut,pdmp_po_kwut,pin_po_kwut
30745,2,0.754131,-0.357622,0.608978,1.208102,1.063071,1.036922,2.638846,1.230662,-1.439695,26.091471,-0.779482,2.485585,3.380611
6218,5,0.063349,-0.323664,0.502491,1.148399,1.053751,1.048605,0.415318,0.778262,-1.552957,19.375011,-1.365418,1.672439,3.107120
21176,1,0.107422,-0.333411,-0.097432,1.083600,1.007616,1.075522,-0.990991,0.188485,-1.794587,5.808584,-1.878790,0.179217,2.064885
20296,1,0.758094,-0.449904,0.807603,1.249683,1.051666,1.002957,1.159932,1.115530,-1.158134,47.312283,-1.435539,2.817796,4.469747
10020,4,0.931833,-0.244196,0.572156,1.160611,1.079648,1.041277,1.417187,0.547345,-1.493009,26.833326,-1.149813,1.994170,3.213593
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33062,6,0.752040,-0.450206,0.478991,1.146872,1.009033,1.060395,1.811131,1.141111,-1.596953,22.827461,-1.967635,1.470562,3.527074
25630,2,0.225919,-0.294887,0.718750,1.223773,1.067474,1.005182,0.612071,0.753187,-1.276109,30.984837,-0.237244,2.900426,3.257240
4647,5,-0.134290,-0.384222,0.560237,1.170411,1.047444,1.049498,-0.073744,0.874596,-1.493403,26.943375,-1.299614,2.200275,3.600426
26960,2,0.219669,-0.328324,0.671677,1.222390,1.049287,1.026232,0.740775,0.801082,-1.359085,32.912922,-0.796293,2.587804,3.526629


In [18]:
y_test

24969     3
4462      1
19651     5
21270     2
27728    10
         ..
18887     1
22889     5
22205     6
1353      6
10319     8
Name: fault_class, Length: 6829, dtype: int32

In [19]:
lr_list = [0.05, 0.075, 0.1, 0.25, 0.5, 0.75, 1]

for learning_rate in lr_list:
    gb_clf = GradientBoostingClassifier(n_estimators=20, learning_rate=learning_rate, max_features=2, max_depth=2, random_state=0)
    gb_clf.fit(X_train, y_train)

    print("Learning rate: ", learning_rate)
    print("Accuracy score (training): {0:.3f}".format(gb_clf.score(X_train, y_train)))
    print("Accuracy score (validation): {0:.3f}".format(gb_clf.score(X_test, y_test)))


Learning rate:  0.05
Accuracy score (training): 0.705
Accuracy score (validation): 0.696
Learning rate:  0.075
Accuracy score (training): 0.737
Accuracy score (validation): 0.724
Learning rate:  0.1
Accuracy score (training): 0.759
Accuracy score (validation): 0.744
Learning rate:  0.25
Accuracy score (training): 0.839
Accuracy score (validation): 0.825
Learning rate:  0.5
Accuracy score (training): 0.885
Accuracy score (validation): 0.861
Learning rate:  0.75
Accuracy score (training): 0.904
Accuracy score (validation): 0.892
Learning rate:  1
Accuracy score (training): 0.905
Accuracy score (validation): 0.884


In [24]:
xgb_clf = XGBClassifier()
xgb_clf.fit(X_train, y_train - 1)

score = xgb_clf.score(X_test, y_test - 1)
print(score)


0.9704202665104701
